# Lecture2


## Backpropagation
 We mentioned in lecture 1 that PyTorch can compute the gradient to any function. In this case we have indirections.
 PyTorch builds a computation graph and uses the chain rule
 we introduce with an example

![Comp-graph-1](comp-graph-1.png)

![Comp-graph-2](comp-graph-2.png)

![backprop](backprop.png)

In [5]:
import torch
a=torch.tensor(2.,requires_grad=True)
b=torch.tensor(4.,requires_grad=True)
c=a+b
c.retain_grad()
d=torch.log(a)+
d.retain_grad()
e=c*d
e.backward()

In [6]:
print(a.grad,b.grad)
print(c.grad,d.grad)

tensor(10.6562) tensor(4.1640)
tensor(2.) tensor(6.)


In [6]:
import math
6/math.log(2)

8.656170245333781